In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
from unstructured.partition.pdf import partition_pdf

file_path = '/Users/kamal/Desktop/AgenticAI/Uploads/Attention Is All You Need.pdf'

chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,           
    strategy="hi_res",                     

    extract_image_block_types=["Image", "Table"],   
    image_output_dir_path="/Users/kamal/Desktop/AgenticAI/Side Projects/RAG Assignment/Images",   

    extract_image_block_to_payload=True,   

    chunking_strategy="by_title",          
    max_characters=10000,                  
    combine_text_under_n_chars=2000,       
    new_after_n_chars=6000
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/kamal/miniconda3/envs/agenticaienv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox mi

In [4]:
chunks, len(chunks)

([<unstructured.documents.elements.CompositeElement at 0x17ff072e0>,
 12)

In [5]:
chunks[8]

In [6]:
import os
import uuid
import base64
from collections import Counter
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
image_output_dir = "/Users/kamal/Desktop/AgenticAI/Side Projects/RAG Assignment/Images"
os.makedirs(image_output_dir, exist_ok=True)

documents = []

for chunk in chunks:
    chunk_id = str(uuid.uuid4())
    content_pieces, page_numbers, image_paths = [], [], []

    for idx, elem in enumerate(getattr(chunk.metadata, 'orig_elements', [])):
        if (text := getattr(elem, 'text', None)):
            content_pieces.append(text)

        if (page := getattr(elem.metadata, 'page_number', None)) is not None:
            page_numbers.append(page)

        if (b64 := getattr(elem.metadata, 'image_base64', None)):
            try:
                img_bytes = base64.b64decode(b64)
                img_path = os.path.join(image_output_dir, f"{chunk_id}_{idx}.png")
                with open(img_path, "wb") as f:
                    f.write(img_bytes)
                image_paths.append(img_path)
            except Exception as e:
                print(f"Image decode error in chunk {chunk_id}: {e}")

    content = "\n".join(content_pieces)
    page_number = Counter(page_numbers).most_common(1)[0][0] if page_numbers else -1

    try:
        embedding_response = openai.embeddings.create(
            input=content,
            model="text-embedding-3-small"
        )
        embedding = embedding_response.data[0].embedding
    except Exception as e:
        print(f"Embedding error in chunk {chunk_id}: {e}")
        embedding = [0.0] * 1536

    doc = {
        "id": chunk_id,
        "embedding": embedding,
        "content": content,
        "type": getattr(chunk, 'type', "Composite"),
        "section_title": getattr(chunk, 'section_title', "") or "",
        "page_number": page_number,
        "image_path": image_paths,
    }

    documents.append(doc)


In [9]:
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType, list_collections

# 1. Connect to Milvus
connections.connect(alias="default", host="localhost", port="19530")

# 2. Define schema
fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=36),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=20),
    FieldSchema(name="section_title", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="page_number", dtype=DataType.INT64),
    FieldSchema(name="image_path", dtype=DataType.VARCHAR, max_length=1024),
]

schema = CollectionSchema(fields, description="RAG Assignment")
collection_name = "rag_assignment_ipynb"

# 3. Create collection if not exists
if collection_name not in list_collections():
    collection = Collection(name=collection_name, schema=schema)
else:
    collection = Collection(name=collection_name)

# 4. Prepare and insert data
data_to_insert = [
    [doc["id"] for doc in documents],
    [doc["embedding"] for doc in documents],
    [doc["content"][:4096] for doc in documents],
    [doc["type"] for doc in documents],
    [doc["section_title"][:256] for doc in documents],
    [doc["page_number"] for doc in documents],
    [",".join(doc["image_path"])[:1024] for doc in documents],
]

collection.insert(data_to_insert)
collection.flush()

index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128}
}

collection.create_index(field_name="embedding", index_params=index_params)

collection.load()

print(f"Inserted {len(documents)} documents into Milvus collection '{collection_name}'.")

Inserted 12 documents into Milvus collection 'rag_assignment_ipynb'.


In [10]:
from langchain.vectorstores import Milvus
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Milvus(
    embedding_function=embeddings,
    collection_name="rag_assignment",
    connection_args={"host": "localhost", "port": "19530"},
    text_field="content",
    vector_field="embedding",     
    index_params={"index_type": "IVF_FLAT", "metric_type": "COSINE"},
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

/var/folders/_r/5tgc_pmx6z36cqlgkrdf1k_r0000gr/T/ipykernel_96146/3867490706.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
/var/folders/_r/5tgc_pmx6z36cqlgkrdf1k_r0000gr/T/ipykernel_96146/3867490706.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vector_store = Milvus(


In [12]:
pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.schema import Document

bm25_documents = [
    Document(
        page_content=doc["content"],
        metadata={
            "id": doc["id"],
            "type": doc["type"],
            "section_title": doc["section_title"],
            "page_number": doc["page_number"],
            "image_path": doc["image_path"]
        }
    )
    for doc in documents
]

from langchain.retrievers import BM25Retriever

sparse_retriever = BM25Retriever.from_documents(bm25_documents)
sparse_retriever.k = 3

In [33]:
sparse_retriever.invoke("Multi-Head Attention")

[Document(metadata={'id': '42887f8d-8b6e-4d27-b359-ad8c29878179', 'type': 'Composite', 'section_title': '', 'page_number': 3, 'image_path': ['/Users/kamal/Desktop/AgenticAI/Side Projects/RAG Assignment/Images/42887f8d-8b6e-4d27-b359-ad8c29878179_3.png']}, page_content='3 Model Architecture\nMost competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35]. Here, the encoder maps an input sequence of symbol representations (x1,...,xn) to a sequence of continuous representations z = (z1,...,zn). Given z, the decoder then generates an output sequence (y1,...,ym) of symbols one element at a time. At each step the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next.\n2\nOutput Probabilities Add & Norm Feed Forward Add & Norm Multi-Head Attention a, Add & Norm Add & Norm Feed Forward Nx | Cag Norm) Add & Norm Masked Multi-Head Multi-Head Attention Attention Se a, Lt Positional Positional Encodi

In [28]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[sparse_retriever, retriever],  # [sparse, dense]
    weights=[0.7, 0.3]  # alpha = 0.5
)


In [29]:
query = "Multi-Head Attention"
top_k_docs = ensemble_retriever.invoke(query)

# Use your reranking pipeline
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L6-v2')

pairs = [(query, doc.page_content) for doc in top_k_docs]
scores = cross_encoder.predict(pairs)

scored_docs = sorted(zip(top_k_docs, scores), key=lambda x: x[1], reverse=True)
ranked_docs = [doc for doc, score in scored_docs]

In [30]:
ranked_docs

[Document(metadata={'id': '9e459a01-1c9e-442f-912c-e5e695651ae6', 'type': 'Composite', 'section_title': '', 'page_number': 4, 'image_path': ['/Users/kamal/Desktop/AgenticAI/Side Projects/RAG Assignment/Images/9e459a01-1c9e-442f-912c-e5e695651ae6_5.png', '/Users/kamal/Desktop/AgenticAI/Side Projects/RAG Assignment/Images/9e459a01-1c9e-442f-912c-e5e695651ae6_6.png']}, page_content='3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3\nScaled Dot-Product Attention\nMulti-Head Attention\nLinear\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attent

In [31]:
from langchain import hub
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from IPython.display import Image, display
import os

# Load prompt and LLM
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# Prepare top 2 documents
top_docs = ranked_docs[:2]
context = "\n\n".join([doc.page_content for doc in top_docs])

# Run RAG synthesis
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"context": context, "question": query})

# Print response
print("🔍 Synthesized Answer:\n")
print(response)


🔍 Synthesized Answer:

Multi-Head Attention enhances the attention mechanism by projecting queries, keys, and values into multiple subspaces and performing attention in parallel across these projections. This allows the model to jointly attend to different representation subspaces, improving its ability to capture diverse information. In practice, it consists of several attention heads that are concatenated and projected to produce the final output.


In [32]:
context

'3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3\nScaled Dot-Product Attention\nMulti-Head Attention\nLinear\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the √ dk, and apply a softmax function to obtain the weights on the query with all keys, divide each by values.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together into a m